In [1]:
import osmnx as ox
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster

In [2]:
price_today= pd.read_excel('price_today.xlsx')


In [3]:
geolocator = Nominatim(user_agent="My-App")
location = geolocator.geocode("Plaza Maestro Tarrega, 8, Alcobendas") 

print((location.latitude, location.longitude))

(40.5492094, -3.6446537)


In [4]:
type(location)

geopy.location.Location

In [5]:
price_today = price_today.sort_values('Distance')
price_head=price_today.head(3)
price_head.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 6691 to 6703
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Provincia                      3 non-null      object 
 1   Municipio                      3 non-null      object 
 2   Localidad                      3 non-null      object 
 3   Código postal                  3 non-null      int64  
 4   Dirección                      3 non-null      object 
 5   Longitud                       3 non-null      float64
 6   Latitud                        3 non-null      float64
 7   Precio gasolina 95 E5          3 non-null      object 
 8   Precio gasolina 95 E5 Premium  3 non-null      object 
 9   Precio gasolina 98 E5          3 non-null      object 
 10  Precio gasóleo A               3 non-null      object 
 11  Precio gasóleo Premium         3 non-null      object 
 12  Rótulo                         3 non-null      o

In [6]:
#madrid_street = ox.graph_from_place('Madrid, España', network_type = 'drive') 

#blr_street_projected = ox.project_graph(madrid_street)
 
#ox.plot_graph(blr_street_projected)

In [7]:
# tipos de tiles:

# - CartoDB dark_matter
# - Stamen Toner

In [8]:
#probar las páginas web.
# con folium voy creando mapas
# con OSMox 

In [9]:
#folium_map = folium.Map(location=[40.4167047, -3.7035825],  width=1000, height=1000,
                        #zoom_start=10,
                        #tiles='cartodbpositron')

#folium_map


In [10]:
#MarkerCluster(data=list(zip(price_head['Latitud'].values, price_head['Longitud'].values))).add_to(folium_map)
#folium.LayerControl().add_to(folium_map)
#folium_map



In [11]:

m = folium.Map(location=[40.546376,-3.658541], zoom_start=15)


marker_cluster = MarkerCluster().add_to(m)

folium.Marker(
    location=[price_head.iloc[0]['Latitud'],price_head.iloc[0]['Longitud']],
    popup="Add popup text here.",
    icon=folium.Icon(color="green", icon="ok-sign"),
).add_to(marker_cluster)

folium.Marker(
    location=[price_head.iloc[1]['Latitud'],price_head.iloc[1]['Longitud']],
    popup="Add popup text here.",
    icon=folium.Icon(color="green",  icon="ok-sign"),
).add_to(marker_cluster)

folium.Marker(
    location=[price_head.iloc[2]['Latitud'],price_head.iloc[2]['Longitud']],
    popup="Add popup text here.",
    icon=folium.Icon(color="green",  icon="ok-sign"),
).add_to(marker_cluster)

folium.Marker(
    location=[price_head.iloc[0]['Latitud_partida'],price_head.iloc[2]['Longitud_partida']],
    popup="Add popup text here.",
    icon=folium.Icon(color="lightgray",  icon="home"),
).add_to(marker_cluster)

m

In [12]:
html=f"""
    <h> {price_head['Rótulo']}</h>
    <p><b>Prices:</b></p>
        <li> Gasolina 95:{price_head['Precio gasolina 95 E5']}€</li>
        <li> Diésel:{price_head['Precio gasóleo A']}€</li>
    </ul> 
    </p>
    <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p>
    """
iframe = folium.IFrame(html=html, width=200, height=200)
popup = folium.Popup(iframe, max_width=2650)
popup

In [13]:
#FUNCION BUENA
mapa_loop = folium.Map(location=[40.546376,-3.638541], zoom_start=15)

marker_cluster = MarkerCluster().add_to(mapa_loop)

def ubi_gasolinera(price_head):
    for i in range(0,len(price_head)):
        html=f"""
            <h> {price_head.iloc[i]['Rótulo']}</h>
            <p><b>Prices:</b></p>
                <li> Gasolina 95:{price_head.iloc[i]['Precio gasolina 95 E5']}€</li>
                <li> Diésel:{price_head.iloc[i]['Precio gasóleo A']}€</li>
            </ul> 
            </p>
            <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p>
            """
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        
        folium.Marker(
                location=[price_head.iloc[i]['Latitud'],price_head.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="green",  icon="ok-sign"),
        ).add_to(marker_cluster)
    return mapa_loop

def ubi_actual(price_head):
    html=f"""
        <h><b> Actualmente te encuentras aquí </b></h>
        """
    iframe = folium.IFrame(html=html, width=150, height=60)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[price_head.iloc[0]['Latitud_partida'],price_head.iloc[0]['Longitud_partida']],
        popup=popup,
        icon=folium.Icon(color="lightgray",  icon="home"),
).add_to(marker_cluster)  
    return mapa_loop
    
ubi_gasolinera(price_head)
ubi_actual(price_head)

In [14]:
def statistical_quartiles(dataset, column, percentile):
    lst = dataset[column].tolist()
    data = sorted(lst)
    if percentile != 100:
        index = int((len(data))*percentile/100)
        #Par, si el resto es 0.  
        if len(lst) % 2 != 0:
            return data[index]

        #Impar, si el resto != de 0.
        else:
            return (data[index - 1] + data[index]) / 2
        
    elif percentile == 100:
        index = int(len(data))-1  
        return data[index]

In [15]:
df_colores = price_head.iloc[:,[7,10]]
df_colores


,Precio gasolina 95 E5,Precio gasóleo A
6691,"1,875","1,880"
6690,"1,875","1,880"
6703,"1,909","1,949"


Paso a float

In [16]:
df_colores['Precio gasolina 95 E5'] = df_colores['Precio gasolina 95 E5'].apply(lambda x: x.replace(',','.'))
df_colores['Precio gasóleo A'] = df_colores['Precio gasóleo A'].apply(lambda x: x.replace(',','.'))
df_colores['Precio gasolina 95 E5']=df_colores['Precio gasolina 95 E5'].astype('float64')
df_colores['Precio gasóleo A']=df_colores['Precio gasóleo A'].astype('float64')


percentil_50=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 50) #amarillo
percentil_75=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 75) #rojo
percentil_25=statistical_quartiles(df_colores,'Precio gasolina 95 E5', 25) #verde

#df_colores
df_colores.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 6691 to 6703
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Precio gasolina 95 E5  3 non-null      float64
 1   Precio gasóleo A       3 non-null      float64
dtypes: float64(2)
memory usage: 72.0 bytes


/Users/Andrea/miniconda3/envs/proj-final/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/Andrea/miniconda3/envs/proj-final/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Andrea/miniconda3/envs/proj-final/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [17]:
#float(percentil_50)
#float(percentil_75)
#float(percentil_25)
#type(percentil_50)
#type(percentil_75)
#type(percentil_25)

price_head['Precio gasolina 95 E5'] = price_head['Precio gasolina 95 E5'].apply(lambda x: x.replace(',','.'))
price_head['Precio gasóleo A'] = price_head['Precio gasóleo A'].apply(lambda x: x.replace(',','.'))

price_head['Precio gasóleo A']=price_head['Precio gasóleo A'].astype('float64')
price_head['Precio gasolina 95 E5']=price_head['Precio gasolina 95 E5'].astype('float64')

#price_head.info()
#type(percentil_25)

/Users/Andrea/miniconda3/envs/proj-final/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Andrea/miniconda3/envs/proj-final/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/Andrea/miniconda3/envs/proj-final/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [31]:
bins = [0,percentil_25,percentil_75,100]
colores = ['verde','amarillo','rojo']
price_head['Colores']= pd.cut(price_head['Precio gasolina 95 E5'],bins,labels = colores)

/Users/Andrea/miniconda3/envs/proj-final/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
price_head


,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,...,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish,Distance,Colores
6691,MADRID,Madrid,MADRID,28028,"CALLE BERLIN, S/N, S/N",-3.664333,40.441222,1.875,No disponible,No disponible,...,"1,941",CEPSA,L-D: 07:00-21:30,25-04-2022,40.440274,-3.661032,POINT (4501790.72035241 -407821.8017149498),POINT (4501896.240097732 -408190.0417713508),383.060251,verde
6690,MADRID,Madrid,MADRID,28028,"AVENIDA DE LOS TOREROS, S/N",-3.661944,40.434306,1.875,No disponible,No disponible,...,"1,941",CEPSA,L-S: 07:00-21:00; D: 9:00-14:00,25-04-2022,40.440274,-3.661032,POINT (4501790.72035241 -407821.8017149498),POINT (4501126.354499406 -407923.5550565211),672.112884,verde
6703,MADRID,Madrid,MADRID,28002,"CALLE CORAZON DE MARIA, 11",-3.669500,40.441611,1.909,No disponible,No disponible,...,"1,999",CEPSA,L-V: 07:00-19:00; S: 09:00-15:00,25-04-2022,40.440274,-3.661032,POINT (4501790.72035241 -407821.8017149498),POINT (4501939.543379651 -408766.409570259),956.259533,amarillo


In [35]:
percentil_25

1.875

In [ ]:
m = folium.Map(location=[40.546376,-3.658541], zoom_start=15, control_scale=True)


marker_cluster = MarkerCluster().add_to(m)

html=f"""
    <h> {price_head.iloc[1]['Rótulo']}</h>
    <p><b>Prices:</b></p>
        <li> Gasolina 95:{price_head.iloc[1]['Precio gasolina 95 E5']}€</li>
        <li> Diésel:{price_head.iloc[1]['Precio gasóleo A']}€</li>
    </ul> 
    </p>
    <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p>
    """

iframe = folium.IFrame(html=html, width=200, height=200)
popup = folium.Popup(iframe, max_width=2650)


    folium.Marker(
        location=[price_head.iloc[1]['Latitud'],price_head.iloc[1]['Longitud']],
        popup= popup,
        icon=folium.Icon(color = columna_color,icon="fa-car", prefix="fa"),
        #icon = folium.features.CustomIcon('REPSOL.png', icon_size=(70, 50)),
    ).add_to(marker_cluster)

# hacer un diccionario con la columnas de marca y el value sea la foto. hacer icon= columna[i] y luego que me de el value
# tags: <b> es negrita, <p> es pequeña, <h> es grande y se ponen números para aumentarlo


m

In [49]:
mapa_loop = folium.Map(location=[40.546376,-3.638541], zoom_start=15)

#marker_cluster = MarkerCluster().add_to(mapa_loop)

def ubi_gasolinera(price_head):
    for i in range(0,len(price_head)):
        html=f"""
            <div style="font-family: arial; color: black">
            <h><b> {price_head.iloc[i]['Rótulo']}</h></b>
            <p>Prices:</p>
                <li> Gasolina 95:{price_head.iloc[i]['Precio gasolina 95 E5']} €</li>
                <li> Diésel:{price_head.iloc[i]['Precio gasóleo A']} €</li>
            <p><b>Dirección:</b></p>
            <p>{price_head.iloc[i]['Dirección']}</p>
            </ul> 
            </p>
            <img src="my_plot_name.png">
            <p>And that's a <a href="https://www.python-graph-gallery.com">link</a></p></div>
            """
        
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        
        if price_head.iloc[i]['Colores']== 'verde':
            folium.Marker(
                location=[price_head.iloc[i]['Latitud'],price_head.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="green",  icon="ok-sign"),
            ).add_to(mapa_loop)
        elif price_head.iloc[i]['Colores']== 'amarillo':
            folium.Marker(
                location=[price_head.iloc[i]['Latitud'],price_head.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="blue",  icon="ok-sign"),
            ).add_to(mapa_loop)
            
        elif price_head.iloc[i]['Colores']== 'rojo':
            folium.Marker(
                location=[price_head.iloc[i]['Latitud'],price_head.iloc[i]['Longitud']],
                popup=popup,
                icon=folium.Icon(color="red",  icon="ok-sign"),
            ).add_to(mapa_loop)
            
    return mapa_loop

def ubi_actual(price_head):
    html=f"""
        <div style="font-family: times new roman; color: green">
        <h><b> Actualmente te encuentras aquí </b></h></div>
        """
    iframe = folium.IFrame(html=html, width=150, height=60)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[price_head.iloc[0]['Latitud_partida'],price_head.iloc[0]['Longitud_partida']],
        popup=popup,
        icon=folium.Icon(color="cadetblue",  icon="home"),
).add_to(mapa_loop)  
    return mapa_loop
    
ubi_gasolinera(price_head)
ubi_actual(price_head)

In [45]:

    
in_2008 = price_head['Colores'] == 'amarillo'
in_2008

6691    False
6690    False
6703     True
Name: Colores, dtype: bool